In [1]:
import numpy as np
import pandas as pd
import mat73
from tqdm import tqdm
import os

In [ ]:
# Assuming signal is your original signal stored as a numpy array
# signal = np.array([...])
sampling_rate = 128
time_step = 8
window_length = 3  # in seconds
num_points = sampling_rate * window_length
threshold = 0.4  # 0.75 0.5 0.4
cluster_len = 2  # 10 8 5 3

In [ ]:
def hard_mine(df):
    # Correcting the column name and identifying false positives
    column_name = df.columns[0]
    false_positives = df[df[column_name] > threshold]

    # Display the number of false positives and their first few rows
    num_false_positives = len(false_positives)
    false_positives.head(), num_false_positives

    # Find clusters of false positives
    clusters = []
    current_cluster = []

    # Iterate over the false positives
    for idx in false_positives.index:
        # If current_cluster is empty or the current index is consecutive to the last index in current_cluster
        if not current_cluster or idx == current_cluster[-1] + 1:
            current_cluster.append(idx)
        else:
            # If the current index is not consecutive, check if the current cluster is valid (has at least 8 false positives)
            if len(current_cluster) >= cluster_len:
                clusters.append(current_cluster)
            # Reset current_cluster and start a new one
            current_cluster = [idx]

    # Check for the last cluster
    if len(current_cluster) >= cluster_len:
        clusters.append(current_cluster)

    # Display the number of clusters and the indices of the first few clusters
    num_clusters = len(clusters)
    # Extract the index of the highest prediction value from each cluster
    highest_prediction_indices = []
    bonobo_idx = []

    for cluster in clusters:
        max_index = df.loc[cluster][column_name].idxmax()
        highest_prediction_indices.append(max_index)

    filtered_indices = []
    for idx in sorted(highest_prediction_indices):
        if not filtered_indices or (idx - filtered_indices[-1] > num_points):
            filtered_indices.append(idx)
        else:
            if idx - filtered_indices[-1] <= num_points:
                last_idx = filtered_indices.pop()  # Store the popped value if needed
    return filtered_indices

In [ ]:
path_controls = os.path.join("your_path/SpikeNet2/controlset.csv")
controls = pd.read_csv(path_controls)
# train_controls = controls[controls['Mode']=='Train']
train_controls = controls

for eeg_file in tqdm(train_controls.EEG_index):
    df_path = "your_path/SpikeNet2/Models/SpikeNet2/con_hardmine/" + eeg_file + ".csv"
    signal_path = "your_path/Bonobo_data/" + eeg_file + ".mat"
    df = pd.read_csv(df_path)
    filtered_indices = hard_mine(df)
    signal = mat73.loadmat(signal_path)["data"]  # 128 sample rate
    signal = signal.transpose(1, 0)
    start_times = filtered_indices  # 16 sample rate

    # Calculate start indices
    start_indices = [int(time * time_step) for time in start_times]

    # Extract signal segments
    segments = [signal[start : start + num_points] for start in start_indices]
    for idx, seg in zip(start_indices, segments):
        path = str(
            "your_path/SpikeNet2/Models/SpikeNet2/hardmine_npy_round2/"
            + eeg_file
            + "_"
            + str(idx)
            + ".npy"
        )
        seg = seg.transpose(1, 0)
        # print(seg.shape)
        np.save(path, seg)

In [ ]:
def count_files_in_directory(directory_path):
    items = os.listdir(directory_path)

    return len(
        [item for item in items if os.path.isfile(os.path.join(directory_path, item))]
    )

In [ ]:
count_files_in_directory("your_path/SpikeNet2/Models/SpikeNet2/hardmine_npy_round2")

# read csv
df = pd.read_csv("your_path/SpikeNet2/hard_mining_round1.csv")

# Get the names of all .npy files in the folder
npy_files = [
    f
    for f in os.listdir("your_path/SpikeNet2/Models/SpikeNet2/hardmine_npy_round2")
    if f.endswith(".npy")
]

# Extract file name part
event_files = [f[:-4] for f in npy_files]  # remove.npy
eeg_files = [
    f.split("_")[:-1] for f in event_files
]  # # Use '_' as delimiter and remove the last part
eeg_files = ["_".join(f) for f in eeg_files]
# Create a new DataFrame to store filenames and other information
new_data = {
    "event_file": event_files,
    "eeg_file": eeg_files,
    "total_votes_received": [3] * len(event_files),
    "fraction_of_yes": [0] * len(event_files),
    "Mode": ["Train"] * len(event_files),
}

new_df = pd.DataFrame(new_data)

# Add new data to the original DataFrame
df = pd.concat([df, new_df], ignore_index=True)
df.to_csv("your_path/SpikeNet2/hard_mining_round2.csv", index=False)